<a href="https://colab.research.google.com/github/ayush12gupta/CNN_models/blob/master/ResNet/ResNet101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np


In [0]:
class block(nn.Module):
    def __init__(self, inp, out, strides=1):
      super().__init__()

      self.conv1 = nn.Conv2d(inp, out, 3, padding=1, stride=strides, bias=False)
      self.bn1 = nn.BatchNorm2d(out)
      self.conv2 = nn.Conv2d(out, out, 3, padding=1, bias=False) # For same size of layer as the previour one
      self.bn2 = nn.BatchNorm2d(out)

      self.shortcut = nn.Sequential()
      if strides!=1 or inp!=out:
        self.shortcut = nn.Sequential(
            nn.Conv2d(inp, out,1,stride = strides,bias=False),
            nn.BatchNorm2d(out)
        )

    def forward(self,x):
        iden = x
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.bn2(self.conv2(x))
        x += self.shortcut(iden)  # Making identity connection
        
        return F.relu(x)

In [0]:
class bottleneck(nn.Module):
  def __init__(self, inp, out, strides=1):
    super().__init__()
    
    self.conv1 = nn.Conv2d(inp, out,1, stride=1,bias=False)
    self.bn1 = nn.BatchNorm2d(out)
    self.conv2 = nn.Conv2d(out, out, 3, stride=strides, padding=1, bias=False) 
    self.bn2 = nn.BatchNorm2d(out)
    self.conv3 = nn.Conv2d(out, 4*out, 1, stride=1,bias=False)
    self.bn3 = nn.BatchNorm2d(4*out)
    
    self.shortcut = nn.Sequential()
    if strides!=1 or inp!=out*4:
        self.shortcut = nn.Sequential(
            nn.Conv2d(inp, out*4,1,stride = strides,bias=False),
            nn.BatchNorm2d(out*4)
        )


  def forward(self,x):
      iden = x
      x = F.relu(self.bn1(self.conv1(x)))
      x = F.relu(self.bn2(self.conv2(x)))
      x = self.bn3(self.conv3(x))
      #print(x.shape," s ",self.shortcut(iden).shape)
      x += self.shortcut(iden)

      return F.relu(x) 

In [0]:
class ResNet(nn.Module):
    def __init__(self, block, n,classnum=10):

        super().__init__()
        if block==bottleneck:
          self.exp=4
        if block==block:
          self.exp=1  
        self.inp = 64
        self.cov1 = nn.Conv2d(3, 64, 3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.cb1 = self.covBlock(block, 64,n[0],stride=1)
        self.cb2 = self.covBlock(block, 128,n[1],stride=2)
        self.cb3 = self.covBlock(block, 256,n[2],stride=2)
        self.cb4 = self.covBlock(block, 512,n[3],stride=2)
        self.fc = nn.Linear(512*self.exp, classnum)

    def covBlock(self,block,out,n,stride):

        # if stride!=1:
        #   inp,out = size
        # else:
        #   inp,out = size,size

        layer = []
        layer.append(block(self.inp,out,stride))
        self.inp = out*self.exp
        #self.inp = self.inp*4
        for i in range(n-1):
          layer.append(block(self.inp,out))
          self.inp = out*self.exp
        cb = nn.Sequential(*layer)
        return cb  

    def forward(self, x):

        x = F.relu(self.bn1(self.cov1(x))) 
        x = self.cb1(x)
        x = self.cb2(x)
        x = self.cb3(x)
        x = self.cb4(x)
        x = F.avg_pool2d(x, 4)
        x = x.view(x.size(0), -1)  
        # x = self.avgpool(x)

        # x = x.view(-1, res)
        x = self.fc(x)
        x = F.softmax(x,dim=1)

        return x            

In [11]:
model = ResNet(bottleneck,[3,4,24,3])
print(model)

ResNet(
  (cov1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (cb1): Sequential(
    (0): bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (1): bottleneck(
      (conv1): Conv2d(64, 6

In [13]:
batch_size = 64

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((.5, .5, .5), (.5, .5, .5))])

trainset = torchvision.datasets.CIFAR10(root = './data', download = True, train = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root = './data', download = True, train = False, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = True, num_workers=2) # num workers is used to pre process data

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [0]:
model = ResNet(bottleneck, [3,4,24,3]).cuda() #ResNet101
model = ResNet(block, [3,4,6,3]).cuda()
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate,weight_decay=0.0001)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30,50,70,80], gamma=0.1)
device = 'cuda'

In [21]:
model.train()
print('Training Started')
losses = []
for epoch in range(100):

    Tloss,losss = 0., 0.
    correct, total = 0., 0.
    correct_epoch, total_epoch = 0., 0.
    scheduler.step()
    for i,data in enumerate(trainloader):

        inp,lab = data[0].to(device),data[1].to(device)

        optimizer.zero_grad()

        out = model(inp)
        loss = criterion(out,lab)
        
        loss.backward()
        optimizer.step()
        
        Tloss += loss.item()
        losss += loss.item()
        c = (torch.argmax(out,1)==lab)
        correct+=torch.sum(c)
        total += batch_size
        correct_epoch += torch.sum(c)
        total_epoch += batch_size
        
        if i % 200 == 199:    # print every 200 mini-batches
            #print('[%d, %5d] loss: %.7f accuracy: [%d, %d] (%d %%)' % (epoch + 1, i + 1, running_loss / 200, correct, total, 100*correct/total))
            print('Epoch: %d   loss: %.5f  accuracy: %d / %d (%.3f %%)'%(epoch+1,Tloss/200,correct,total,(correct/total)*100))
            correct = 0.
            total = 0.
            Tloss = 0.
    print('Epoch: %d Completed !!!! accuracy: %d / %d (%.3f %%)'%(epoch+1,correct_epoch,total_epoch,(correct_epoch/total_epoch)*100))
    losses.append(correct_epoch/total_epoch)    

Training Started


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Epoch: 1   loss: 2.25626  accuracy: 2333 / 12800 (18.227 %)
Epoch: 1   loss: 2.21057  accuracy: 3064 / 12800 (23.938 %)
Epoch: 1   loss: 2.19398  accuracy: 3291 / 12800 (25.711 %)
Epoch: 1 Completed !!!! accuracy: 3299 / 11648 (23.951 %)
Epoch: 2   loss: 2.14272  accuracy: 3972 / 12800 (31.031 %)
Epoch: 2   loss: 2.11982  accuracy: 4270 / 12800 (33.359 %)
Epoch: 2   loss: 2.09444  accuracy: 4627 / 12800 (36.148 %)
Epoch: 2 Completed !!!! accuracy: 4322 / 11648 (34.349 %)
Epoch: 3   loss: 2.07471  accuracy: 4846 / 12800 (37.859 %)
Epoch: 3   loss: 2.10036  accuracy: 4537 / 12800 (35.445 %)
Epoch: 3   loss: 2.08959  accuracy: 4674 / 12800 (36.516 %)
Epoch: 3 Completed !!!! accuracy: 4520 / 11648 (37.118 %)
Epoch: 4   loss: 2.05769  accuracy: 5056 / 12800 (39.500 %)
Epoch: 4   loss: 2.04345  accuracy: 5288 / 12800 (41.312 %)
Epoch: 4   loss: 2.03516  accuracy: 5357 / 12800 (41.852 %)
Epoch: 4 Completed !!!! accuracy: 5011 / 11648 (41.384 %)
Epoch: 5   loss: 2.02186  accuracy: 5532 / 12800

In [0]:
torch.save(model.state_dict(), '/content/resnet101')


In [16]:
losses

[tensor(0.2445, device='cuda:0'),
 tensor(0.3343, device='cuda:0'),
 tensor(0.3752, device='cuda:0'),
 tensor(0.4100, device='cuda:0')]

In [24]:
model.eval()
running_loss = 0.0
correct, total = 0, 0
#correct_epoch, total_epoch = 0, 0
for i, data in enumerate(testloader):
    inp,lab = data[0].to(device),data[1].to(device)
    out = model(inp) 
    c = (torch.argmax(out,1)==lab)
    correct+=torch.sum(c)
    total += batch_size
    
print('accuracy: ',100*correct/total,'')    

accuracy:  tensor(80, device='cuda:0') 
